# QDC 2025 Challenge - Track A <br> QOBLIB - Market Split Problem

In this challenge notebook, you’ll learn how to tackle complex combinatorial optimization problems using quantum algorithms. Specifically, we’ll explore the Market Split problem — a challenging resource allocation task that becomes computationally intractable even for relatively small instances.

This notebook is divided into four parts:
- Part I: Learn the background of the Market Split problem, and how to fetch and load problem instances from the [Quantum Optimization Benchmarking Library (QOBLIB) repository](https://git.zib.de/qopt/qoblib-quantum-optimization-benchmarking-library).
- Part II: You’ll then convert the problem into a Quadratic Unconstrained Binary Optimization (QUBO) form using the brand new [Qiskit Optimization Mapper addon](https://github.com/Qiskit/qiskit-addon-opt-mapper) to be solved on a quantum computer.
- Part III: See an example of solving the problem using a [Qiskit Function](https://quantum.cloud.ibm.com/docs/guides/functions) — the [Kipu Iskay Optimizer](https://quantum.cloud.ibm.com/docs/guides/kipu-optimization).
- Part IV: Take on the challenge! You’ll tackle a complex instance of the Market Split problem using any quantum methods in your arsenal — whether it’s a Qiskit function, a custom QAOA, or other advanced techniques you’ll encounter throughout the QDC.

Sounds exciting? Let’s get started!

## Table of Contents

* [Imports](#imports)
* [Part I: Understanding and Loading the Market Split Problem (10 points)](#part-I)
* [Part II: QUBO Formulation (10 points)](#part-II)
* [Part III: Solving the problem using Kipu Iskay Optimizer function (0 points)](#part-III)
* [Part IV: Main Challenge — Tackling a Complex Market Split Instance (80 points)](#part-IV)
* [References](#references)
* [Additional Information](#additional-information)
* [Package Versions](#package-versions)

<a id="imports"></a>
## Imports

In [ ]:
# Import common packages
import os
import tempfile
import time
from typing import Tuple, Dict, Optional

import numpy as np
import requests
import matplotlib.pyplot as plt

# Import qiskit classes
from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit_ibm_catalog import QiskitFunctionsCatalog

# Import grading functions
from qc_grader.challenges.qdc_2025.qdc25_lab7 import (
    submit_name,
    grade_kipu_function,
    grade_lab7_ex1,
    grade_lab7_ex2,
    grade_lab7_ex3,
    grade_lab7_ex4,
    grade_lab7_ex5,
    grade_lab7_ex6
)

from qiskit_addon_opt_mapper import OptimizationProblem
from qiskit_addon_opt_mapper.converters import OptimizationProblemToQubo

print("All required libraries imported successfully")
print("Grading functions imported from grader.py")

## Submit Team Name

In [ ]:
# Please use the exact spelling of your team name across teammates and across challenges

team_name = # Add team name string
submit_name(team_name)

<a id="part-I"></a>
## Part I: Understanding and Loading the Market Split Problem (10 points)
### Background: The Market Split Problem

The Market Split problem presents a deceptively simple yet computationally formidable challenge in resource allocation. Consider a company with $m$ products being sold across $n$ different markets, where each market purchases a specific bundle of products (represented by the columns of matrix $A$). The business objective is to partition these markets into two balanced sales regions such that each region receives exactly half the total demand for every product.

**Mathematical Formulation:**
We seek a binary assignment vector $x$, where:
- $x_j = 1$ assigns market $j$ to Region A
- $x_j = 0$ assigns market $j$ to Region B
- The constraint $Ax = b$ must be satisfied, where $b$ represents the target sales (typically half the total demand per product)

**Computational Complexity:**
Despite its straightforward business interpretation, this problem exhibits remarkable computational intractability:
- **Small Scale Failure**: Conventional Mixed Integer Programming solvers fail on instances with as few as 7 products under a timeout of one hour [3].
- **Exponential Growth**: The solution space grows exponentially ($2^n$ possible assignments), making brute force approaches infeasible.

This severe computational barrier, combined with its practical relevance to territory planning and resource allocation, makes the Market Split problem an ideal benchmark for quantum optimization algorithms. The problem instances used in this notebook come from the [Quantum Optimization Benchmarking Library (QOBLIB) repository](https://git.zib.de/qopt/qoblib-quantum-optimization-benchmarking-library) [2], which provides standardized test cases for comparing classical and quantum optimization approaches.

<a id="data-loading-and-problem-format"></a>
### Data Loading and Problem Format

Now let's implement the practical steps to load our problem instance. We have two approaches:

1. **Direct Download**: Use `fetch_marketsplit_data()` to download directly from QOBLIB repository [2]
2. **Manual Download + Parse**: Download the `.dat` file manually from the URL, then parse it with `parse_marketsplit_dat()`

Both approaches require understanding the `.dat` file format to properly extract the problem data.

### Understanding the .dat File Format

Problem instances in QOBLIB are stored in a simple text format. Let's look at the actual content of our target instance [`ms_03_200_177.dat`](https://git.zib.de/qopt/qoblib-quantum-optimization-benchmarking-library/-/blob/main/01-marketsplit/instances/ms_03_200_177.dat?ref_type=heads):

```python
3 20
  60   92  161   53   97    2   75   81    6  139  132   45  108  112  181   93  152  200  164   51 1002
 176  196   41  143    2   88    0   79   10   71   75  148   82  135   34  187   33  155   58   46  879
  68   68  179  173  127  163   48   49   99   78   44   52  173  131   73  198   64   34  158  102 1040
```

**Format Structure:**
- **First line:** `3 20`
  - `3` = number of products (constraints/rows in matrix A)
  - `20` = number of markets (variables/columns in matrix A)

- **Next 3 lines:** Coefficient matrix A and target vector b
  - Each line has 21 numbers: first 20 are row coefficients, last is the target
  - Line 2: `60 92 161 ... 51 | 1002`
    - First 20 numbers: How much of Product 1 each of the 20 markets sells
    - Last number (1002): Target sales for Product 1 in one region
  - Line 3: `176 196 41 ... 46 | 879`
    - Product 2 sales per market and target (879)
  - Line 4: `68 68 179 ... 95 | 1040`
    - Product 3 sales per market and target (1040)

**Business Interpretation:**
- Market 0 sells: 60 units of Product 1, 176 units of Product 2, 68 units of Product 3
- Market 1 sells: 92 units of Product 1, 196 units of Product 2, 68 units of Product 3
- And so on for all 20 markets...
- Goal: Split these 20 markets into two regions where each region gets exactly 1002 units of Product 1, 879 units of Product 2, and 1040 units of Product 3

### Parsing Exercise

Let's implement the function to parse this format. The key challenge is extracting the coefficient matrix `A` and target vector `b` from each line.

<div class="alert alert-block alert-success">
    
<b>Exercise 1: Complete the `parse_marketsplit_dat` Function (8 points)</b>

Your task is to complete the parsing logic that extracts data from each line of the `.dat` file.

**What you need to do:**
- Given a line with `n+1` values (e.g., `60 92 161 ... 51 1002` for `n=20`)
- Extract the first `n` values as a row of matrix `A`
- Extract the last value as the corresponding element of vector `b`

**Hints:**
- Use Python list slicing
- Use negative indexing
- Remember to append to the `A` and `b` lists

</div>

In [ ]:
def parse_marketsplit_dat(filename: str) -> Tuple[np.ndarray, np.ndarray]:
    """
    Parse a market split problem from a .dat file format.

    See the format structure explained above for details on the file layout.

    Parameters
    ----------
    filename : str
        Path to the .dat file containing the market split problem data.

    Returns
    -------
    A : np.ndarray
        Coefficient matrix of shape (m, n) where m is the number of products
        and n is the number of markets.
    b : np.ndarray
        Target vector of shape (m,) containing the target sales per product.
    """
    with open(filename, "r", encoding="utf-8") as f:
        lines = [
            line.strip()
            for line in f
            if line.strip() and not line.startswith("#")
        ]

    if not lines:
        raise ValueError("Empty or invalid .dat file")

    # First line: m n (number of products and markets)
    try:
        m, n = map(int, lines[0].split())
    except (ValueError, IndexError) as e:
        raise ValueError("Invalid file format: first line must contain 'm n' integers") from e

    if len(lines) < m + 1:
        raise ValueError(f"File contains {len(lines)} lines but expected {m + 1} lines")

    # Next m lines: each row of A followed by corresponding element of b
    A = []
    b = []

    for i in range(1, m + 1):
        try:
            values = list(map(int, lines[i].split()))
        except ValueError as e:
            raise ValueError(f"Invalid integer values in line {i + 1}") from e

        if len(values) != n + 1:
            raise ValueError(
                f"Line {i + 1} contains {len(values)} values but expected {n + 1}"
            )

        ### Don't change any code before this line ###

        ### Write your code below here ###
        # TODO: Append the correct values to A list
        # TODO: Extract the target value and append to b list

        ### Don't change any code past this line ###

    return np.array(A, dtype=np.int32), np.array(b, dtype=np.int32)

In [ ]:
# Submit your answer using following code

grade_lab7_ex1(parse_marketsplit_dat) # Expected result type: Callable

### Direct Download Function

This function combines the download and parsing steps:

In [ ]:
def fetch_marketsplit_data(instance_name: str) -> Tuple[Optional[np.ndarray], Optional[np.ndarray]]:
    """
    Fetch market split data directly from the QOBLIB repository.

    This function downloads the .dat file and then uses parse_marketsplit_dat()
    to extract the matrices.

    Parameters
    ----------
    instance_name : str, optional
        Name of the .dat file to fetch from the repository.

    Returns
    -------
    A : np.ndarray or None
        Coefficient matrix of shape (m, n) if successful, None if failed.
    b : np.ndarray or None
        Target vector of shape (m,) if successful, None if failed.
    """
    base_url = (
        "https://git.zib.de/qopt/qoblib-quantum-optimization-benchmarking-library/"
        "-/raw/main/01-marketsplit/instances/"
    )
    url = base_url + instance_name

    try:
        # Fetch the file content with timeout
        response = requests.get(url, timeout=30)
        response.raise_for_status()

        # Save to temporary file
        with tempfile.NamedTemporaryFile(
            mode="w", suffix=".dat", delete=False, encoding="utf-8"
        ) as temp_file:
            temp_file.write(response.text)
            temp_file_path = temp_file.name

        try:
            # Use our parsing function to extract A and b
            A, b = parse_marketsplit_dat(temp_file_path)
            return A, b
        finally:
            # Clean up temporary file
            try:
                os.unlink(temp_file_path)
            except OSError:
                pass  # File cleanup failure is not critical

    except requests.RequestException as e:
        print(f"Error fetching data from repository: {e}")
        return None, None
    except (ValueError, IOError, OSError) as e:
        print(f"Error processing data: {e}")
        return None, None

print("Direct download function defined successfully")
print("\nYou can now use either approach:")
print("  1. fetch_marketsplit_data('ms_03_200_177.dat') - downloads and parses automatically")
print("  2. parse_marketsplit_dat('path/to/file.dat') - parses a manually downloaded file")


Alternatively, you can download the problem instance directly from the QOBLIB repository. 

**Repository URL**: [https://git.zib.de/qopt/qoblib-quantum-optimization-benchmarking-library/-/tree/main/01-marketsplit/instances?ref_type=heads](https://git.zib.de/qopt/qoblib-quantum-optimization-benchmarking-library/-/tree/main/01-marketsplit/instances?ref_type=heads)

<a id="exercise2"></a>
<div class="alert alert-block alert-success">
    
<b>Exercise 2: Load and Analyze Problem Instance (2 points)</b>

Load the Market Split problem instance [`ms_03_200_177.dat`](https://git.zib.de/qopt/qoblib-quantum-optimization-benchmarking-library/-/blob/main/01-marketsplit/instances/ms_03_200_177.dat?ref_type=heads) from the QOBLIB repository [2] and analyze its dimensions.

**Tasks:**
1. Use the `fetch_marketsplit_data()` function to load the problem directly from the repository or download the `.dat` file and use `parse_marketsplit_dat()` to parse it
2. Store the coefficient matrix in variable `A` and target vector in variable `b`
3. Print the problem dimensions and verify it has 3 products and 20 markets

</div>

<div class="alert alert-block alert-info">
    
<b>Tips:</b>

- The function `fetch_marketsplit_data()` takes an `instance_name` parameter
- If you download the instance `.dat` file, use the function `parse_marketsplit_dat()` to parse it. This function takes the path of the file, so make sure you have the correct relative path
- Matrix `A` has shape `(m, n)` where `m` is the number of products and `n` is the number of markets
- Vector `b` contains the target sales for each product (typically half the total demand)

</div>

In [ ]:
### Write your code below here ###

A, b = 

### Don't change any code past this line ###

In [ ]:
# Submit your answer using following code

grade_lab7_ex2(A, b) # Expected result type: numpy.ndarray, numpy.ndarray

In [ ]:
# Verify problem dimensions and display key information
print("Problem Instance Analysis:")
print("=" * 50)
print(f"Coefficient Matrix A: {A.shape[0]} × {A.shape[1]}")
print(f"   → {A.shape[0]} products (constraints)")
print(f"   → {A.shape[1]} markets (decision variables)")
print(f"Target Vector b: {b.shape[0]} elements")
print(f"   → {b.shape[0]} target sales values (half demand per product)")
print(f"Solution Space: 2^{A.shape[1]} = {2**A.shape[1]:,} possible assignments")

# Display the target sales values
print(f"\nTarget Sales (b): {b}")
print("These represent the desired sales per product for each region.")

<a id="part-II"></a>
## Part II: QUBO Formulation (10 points)

<a id="from-constraints-to-qubo"></a>
### From Constraints to QUBO: The Mathematical Transformation

The power of quantum optimization lies in transforming constrained problems into unconstrained quadratic forms [4]. For the Market Split problem, we convert the equality constraints

$$ Ax = b $$

where $x ∈ \{0,1\}^n$, into a QUBO by penalizing constraint violations.

**The Penalty Method:**
Since we need $Ax = b$ to hold exactly, we minimize the squared violation:
$$f(x) = ||Ax - b||^2$$

This equals zero precisely when all constraints are satisfied. Expanding algebraically:
$$f(x) = (Ax - b)^T(Ax - b) = x^T A^T A x - 2b^T A x + b^T b$$

**QUBO Objective:**
Since $b^T b$ is constant, our optimization becomes:
$$\text{minimize} \quad Q(x) = x^T(A^T A)x - 2(A^T b)^T x$$

**Key Insight:** This transformation is exact, not approximate. Equality constraints naturally square into quadratic form without requiring auxiliary variables or penalty parameters — making this formulation mathematically elegant and computationally efficient for quantum solvers [4]. We'll use the `OptimizationProblem` class to define our constrained problem, then convert it to QUBO format using `OptimizationProblemToQubo`. This automatically handles the penalty-based transformation.

<a id="exercise3"></a>
<div class="alert alert-block alert-success">
    
<b>Exercise 3: Convert to QUBO (10 points)</b>

Convert the Market Split problem to QUBO format using the [`qiskit_addon_opt_mapper`](https://github.com/Qiskit/qiskit-addon-opt-mapper) package.

**Tasks:**
1. Create an `OptimizationProblem` object with a name based on `instance_name` (without the `.dat` extension)
2. Add binary variables to the problem (one for each market/column in matrix `A`)
3. Add equality constraints (one for each product/row in matrix `A` with corresponding target value from `b`)
4. Convert the problem to QUBO format with penalty parameter = 1

**What you need to know:**
- Use `OptimizationProblem(name)` to create the problem object
- Use `.binary_var_list(n)` to add n binary variables
- Use `.linear_constraint(coefficients, sense="==", rhs=value)` to add equality constraints
- Use `OptimizationProblemToQubo(penalty=1).convert(problem)` to convert to QUBO

</div>

<div class="alert alert-block alert-info">
    
<b>Tips:</b>

- The number of binary variables equals the number of markets (columns in `A`): `A.shape[1]`
- Loop through each row of `A` and corresponding element of `b` to add constraints
- For constraint i, use `A[idx, :]` to get the coefficients and `b[idx]` for the right-hand side
- Store the resulting QUBO in a variable called `qubo`

</div>

In [ ]:
### Write your code below here ###

# TODO: Create optimization problem
ms = 

# TODO: Add binary variables (one for each market)


# TODO: Add equality constraints (one for each product)


# TODO: Convert to QUBO with penalty parameter
qubo = 

### Don't change any code past this line ###
print("QUBO Conversion Complete:")
print("=" * 50)
print(f"Number of variables: {qubo.get_num_vars()}")
print(f"Constant term: {qubo.objective.constant}")
print(f"Linear terms: {len(qubo.objective.linear.to_dict())}")
print(f"Quadratic terms: {len(qubo.objective.quadratic.to_dict())}")

In [ ]:
# Submit your answer using following code

grade_lab7_ex3(qubo, A, b) # Expected result type: OptimizationProblem, numpy.ndarray, numpy.ndarray

<a id="part-III"></a>
## Part III: Solving the problem using Kipu Iskay Optimizer function (0 points)

In this part, you’ll explore an example of solving the QUBO using a Qiskit Function — the [Kipu Iskay Optimizer](https://quantum.cloud.ibm.com/docs/guides/kipu-optimization). Iskay leverages Kipu's cutting-edge BF-DCQO algorithm [[1](https://journals.aps.org/prresearch/abstract/10.1103/PhysRevResearch.7.L022010),[2](https://doi.org/10.48550/arXiv.2409.04477)], to tackle unconstrained binary optimization problems, for example, in the commonly used QUBO formulation, as well as higher-order (HUBO). BF-DCQO is a non-variational quantum algorithm, which requires fewer computational resources than common variational algorithms, such as QAOA.

As mentioned at the beginning of the notebook, you’re free to approach the final challenge using any quantum method — whether it’s a Qiskit Function, a custom QAOA, or other advanced techniques you’ll encounter during QDC.

This section is optional and not graded, but we highly encourage you to give it a try. You’ll quickly see how the Kipu Iskay Optimizer performs compared to a standard QAOA, and gain valuable insight into its capabilities.

### Loading the Kipu Iskay Quantum Optimizer Function

To use Kipu Quantum's optimization capabilities, we need to establish a connection through the Qiskit Functions Catalog [1]. 

First, initialize the catalog with your credentials:

In [ ]:
# # If you haven't done so already
# # Uncomment the follow code to save the QDC 2025 account provided to you
#
# your_api_key = "deleteThisAndPasteYourAPIKeyHere"
# your_crn = "deleteThisAndPasteYourCRNHere"

# QiskitRuntimeService.save_account(
#     channel='ibm_quantum_platform',
#     token=your_api_key,
#     instance=your_crn,
#     name="qdc-2025"
# )

In [ ]:
# Load the Qiskit Functions Catalog
catalog = QiskitFunctionsCatalog(name="qdc-2025")

# You should see a list of Qiskit Functions available to you
# If you encounter the error `QiskitServerlessException: Credentials couldn't be verified`,
# it means your access is not yet active
display(catalog.list())

print("Successfully initialized Qiskit Functions Catalog")
print("Ready to load quantum optimization functions")

<a id="function-exercise"></a>
<div class="alert alert-block alert-success">
    
<b>Function Exercise: Load the Iskay Quantum Optimizer Function (0 points)</b>

Load the Kipu Quantum Iskay optimizer function from the catalog.

**Tasks:**
1. Determine the correct function name for the Kipu Quantum Iskay Optimizer
2. Use `catalog.load()` to load the function
3. Store the result in variable `iskay_solver`

**What you need to know:**
- The function name follows the format: `"provider/function-name"`
- For Kipu Quantum's Iskay Optimizer, the provider is `"kipu-quantum"`
- The function name is `"iskay-quantum-optimizer"`

</div>

<div class="alert alert-block alert-info">
    
<b>Tips:</b>

- The `catalog.load()` method takes a function name as a string
- Combine the provider and function name with a forward slash: `"provider/function-name"`
- The loaded object will have a `run()` method for executing the optimizer

</div>

In [ ]:
### Write your code below here ###

function_name = ""  # TODO: Specify the Kipu Quantum Iskay function name
iskay_solver = catalog.load(function_name)

In [ ]:
# Submit your answer for grading
print(grade_kipu_function(iskay_solver))

<a id="converting-qubo-to-iskay-format"></a>
### Converting QUBO to Iskay Format

Now we need to convert the QUBO object into the dictionary format required by Kipu Quantum's Iskay Optimizer. 

The `problem` and `problem_type` arguments encode an optimization problem of the form:

$$
\begin{align}
\min_{(x_1, x_2, \ldots, x_n) \in D} C(x_1, x_2, \ldots, x_n) \nonumber
\end{align}
$$
where

$$
C(x_1, ... , x_n) = a + \sum_{i} b_i x_i + \sum_{i, j} c_{i, j} x_i x_j + ... + \sum_{k_1, ..., k_m} g_{k_1, ..., k_m} x_{k_1} ... x_{k_m}
$$

- By choosing `problem_type = "binary"`, you specify that the cost function is in `binary` format, which means that $D = \{0,  1\}^{n}$, as in, the cost function is written in QUBO/HUBO formulation.
- On the other hand, by choosing `problem_type = "spin"`, the cost function is written in Ising formulation, where $D = \{-1, 1\}^{n}$.

The coefficients of the problem should be encoded in a dictionary as follows:
$$
\begin{align} \nonumber
&\texttt{\{} \\ \nonumber
&\texttt{"()"}&: \quad &a, \\ \nonumber
&\texttt{"(i,)"}&: \quad &b_i, \\ \nonumber
&\texttt{"(i, j)"}&: \quad &c_{i, j}, \quad (i \neq j) \\ \nonumber
&\quad  \vdots \\ \nonumber
&\texttt{"(} k_1, ..., k_m  \texttt{)"}&: \quad &g_{k_1, ..., k_m}, \quad (k_1 \neq k_2 \neq \dots \neq k_m) \\ \nonumber
&\texttt{\}}
\end{align}
$$

<div class="alert alert-block alert-warning">
    
<b> Warnings:</b> 

Please note that the keys of the dictionary must be strings containing a valid tuple of non-repeating integers. For binary problems, we know that:

$$
x_i^2 = x_i
$$

for $i=j$ (since $x_i \in \{0,1\}$ means $x_i \cdot x_i = x_i$). So, in your QUBO formulation, if you have both linear contributions $b_i x_i$ and diagonal quadratic contributions $c_{i,i} x_i^2$, these terms must be combined into a single linear coefficient:

**Total linear coefficient for variable $x_i$:** $b_i + c_{i,i}$

This means:
- Linear terms like `"(i, )"` contain: original linear coefficient + diagonal quadratic coefficient
- Diagonal quadratic terms like `"(i, i)"` should **NOT** appear in the final dictionary
- Only off-diagonal quadratic terms like `"(i, j)"` where $i \neq j$ should be included as separate entries

**Example:** If your QUBO has $3x_1 + 2x_1^2 + 4x_1 x_2$, the Iskay dictionary should contain:
- `"(0, )"`: `5.0` (combining $3 + 2 = 5$)
- `"(0, 1)"`: `4.0` (off-diagonal term)

**NOT** separate entries for `"(0, )"`: `3.0` and `"(0, 0)"`: `2.0`.
    
</div>

<a id="exercise4"></a>
<div class="alert alert-block alert-success">
    
<b>Exercise 4: Convert QUBO to Iskay Format (0 points)</b>

Convert the QUBO object into the dictionary format required by Kipu Quantum's Iskay Optimizer.

**Tasks:**
1. Create a dictionary `iskay_input` with the constant term using key `"()"`
2. Add linear coefficients with keys in format `"(i, )"` (note the space before closing parenthesis)
3. Add quadratic coefficients with keys in format `"(i, j)"`

**What you need to know:**
- Constant term: `qubo.objective.constant`
- Linear terms: `qubo.objective.linear` (enumerate to get index and value)
- Quadratic terms: `qubo.objective.quadratic.to_dict()` (returns dictionary with tuple keys)
- All values should be converted to float type
- Use `str(key)` to convert tuple keys to strings for quadratic terms

</div>

<div class="alert alert-block alert-info">
    
<b>Tips:</b>

- Start with an empty dictionary and add the constant term: `iskay_input = {"()": qubo.objective.constant}`
- Use dictionary `.update()` method to add multiple entries at once
- For linear terms, use a dictionary comprehension: `{f"({idx}, )": float(val) for idx, val in enumerate(qubo.objective.linear)}`
- For quadratic terms, use: `{str(key): float(val) for key, val in qubo.objective.quadratic.to_dict().items()}`
- The f-string `f"({idx}, )"` creates the correct format with a space before the closing parenthesis

</div>

In [ ]:
### Write your code below here ###

iskay_input = {}

# TODO: Add constant term
iskay_input = {"()": qubo.objective.constant}

for i in range(qubo.get_num_vars()):
    for j in range(i, qubo.get_num_vars()):
        if i == j:
            # TODO: Add linear term (including diagonal quadratic contribution)
            iskay_input[f"({i}, )"] = 
        else:
            # TODO: Add off-diagonal quadratic term
            iskay_input[f"({i}, {j})"] = float(qubo.objective.quadratic.to_dict().get((i, j)))


In [ ]:
# Submit your answer using following code

grade_lab7_ex4(problem=iskay_input, qubo=qubo)# Expected result type: dict, OptimizationProblem

In [ ]:
print("Iskay Dictionary Format:")
print("=" * 50)
print(f"Total coefficients: {len(iskay_input)}")
print(f"  • Constant term: {iskay_input['()']}")
print(f"  • Linear terms: {sum(1 for k in iskay_input.keys() if k != '()' and ', )' in k)}")
print(f"  • Quadratic terms: {sum(1 for k in iskay_input.keys() if k != '()' and ', )' not in k)}")
print("\nSample coefficients:")

# Get first 10 and last 5 items properly
items = list(iskay_input.items())
first_10 = list(enumerate(items[:10]))
last_5 = list(enumerate(items[-5:], start=len(items)-5))

for i, (key, value) in first_10 + last_5:
    coeff_type = "constant" if key == "()" else "linear" if ", )" in key else "quadratic"
    print(f"  {key}: {value} ({coeff_type})")
print("  ...")
print("\n✓ Problem ready for Iskay optimizer!")

### Understanding the Conversion

We've successfully converted the Market Split problem to QUBO format using the `qiskit_addon_opt_mapper` package. The conversion created:
- An `OptimizationProblem` with binary variables and linear constraints
- A QUBO representation with quadratic, linear, and constant terms
- An Iskay-compatible dictionary for quantum optimization

The `iskay_input` dictionary now contains all the information needed to run the optimization on Kipu Quantum's Iskay Optimizer.

<a id="understanding-iskay-input-parameters"></a>
### Understanding Iskay Input Parameters

The Iskay Optimizer requires several key parameters to effectively solve your optimization problem. Let's examine each parameter and its role in the quantum optimization process:

#### Required Parameters

| Parameter | Type | Description | Example |
|-----------|------|-------------|---------|
| **problem** | `Dict[str, float]` | QUBO coefficients in string-key format | `{"()": -21.0, "(0,4)": 0.5, "(0,1)": 0.5}` |
| **problem_type** | `str` | Format specification: `"binary"` for QUBO or `"spin"` for Ising | `"binary"` |
| **backend_name** | `str` | Target quantum device | `"ibm_boston"` |

#### Essential Concepts

- **Problem Format**: We use `"binary"` since our variables are binary (0/1), representing market assignments.
- **Backend Selection**: Choose between the available QPUs (e.g., `"ibm_boston"`) based on your needs and compute resource instance.
- **QUBO Structure**: Our problem dictionary contains the exact coefficients from the mathematical transformation.

### Advanced Options (Optional)

Iskay provides fine-tuning capabilities through optional parameters. While the defaults work well for most problems, you can customize the behavior for specific requirements:

| Parameter | Type | Default | Description |
|-----------|------|---------|-------------|
| **shots** | `int` | 10000 | Quantum measurements per iteration (higher = more accurate) |
| **num_iterations** | `int` | 10 | Algorithm iterations (more iterations can improve solution quality) |
| **use_session** | `bool` | True | Use IBM sessions for reduced queue times |
| **seed_transpiler** | `int` | None | Set for reproducible quantum circuit compilation |
| **direct_qubit_mapping** | `bool` | False | Map virtual qubits directly to physical qubits |
| **job_tags** | `List[str]` | None | Custom tags for job tracking |
| **preprocessing_level** | `int` | 0 | Problem preprocessing intensity (0-3) - see details below |
| **postprocessing_level** | `int` | 2 | Solution refinement level (0-2) - see details below |
| **transpilation_level** | `int` | 0 | Transpiler optimization trials (0-5) - see details below |
| **transpile_only** | `bool` | False | Analyze circuit optimization without running full execution |

**Preprocessing Levels (0-3)**: Especially important for larger problems that cannot currently fit on the coherence times of the hardware. Higher preprocessing levels achieve shallower circuit depths by approximations in the problem transpilation:
- **Level 0**: Exact, longer circuits
- **Level 1**: Good balance between accuracy and approximation, cutting out only the gates with angles in the lowest 10 percentile
- **Level 2**: Slightly higher approximation, cutting out the gates with angles in the lowest 20 percentile and using `approximation_degree=0.95` in the transpilation
- **Level 3**: Maximum approximation level, cutting out the gates in the lowest 30 percentile and using `approximation_degree=0.90` in the transpilation

**Transpilation Levels (0-5)**: Control the advanced transpiler optimization trials for quantum circuit compilation. This can lead to an increase in classical overhead, and for some cases it might not change the circuit depth. The default value `2` in general leads to the smallest circuit and is relatively fast.
- **Level 0**: Optimization of the decomposed DCQO circuit (layout, routing, scheduling)
- **Level 1**: Optimization of PauliEvolutionGate and then the decomposed DCQO circuit (max_trials=10)
- **Level 2**: Optimization of PauliEvolutionGate and then the decomposed DCQO circuit (max_trials=15)
- **Level 3**: Optimization of PauliEvolutionGate and then the decomposed DCQO circuit (max_trials=20)
- **Level 4**: Optimization of PauliEvolutionGate and then the decomposed DCQO circuit (max_trials=25)
- **Level 5**: Optimization of PauliEvolutionGate and then the decomposed DCQO circuit (max_trials=50)

**Postprocessing Levels (0-2)**: Control how much classical optimization, compensating for bitflip errors with different number of greedy passes of a local search:
- **Level 0**: 1 pass
- **Level 1**: 2 passes
- **Level 2**: 3 passes

**Transpile-Only Mode**: Now available for users who want to analyze circuit optimization without running the full quantum algorithm execution.

### Custom Configuration Example

Here's how you might configure Iskay with different settings:

In [ ]:
# Example: Custom configuration
custom_options = {
    "shots": 15_000,                                        # Higher shot count for better statistics
    "num_iterations": 12,                                   # More iterations for solution refinement
    "preprocessing_level": 1,                               # Light preprocessing for problem simplification
    "postprocessing_level": 2,                              # Maximum postprocessing for solution quality
    "transpilation_level": 3,                               # Using higher transpilation level for circuit optimization
    "seed_transpiler": 42,                                  # Fixed seed for reproducible results
    "job_tags": ["market_split"]                            # Custom tracking tags
}

print("Custom Configuration:")
for key, value in custom_options.items():
    print(f"  {key}: {value}")

### Default Configuration

We'll use a streamlined configuration that balances learning objectives with execution time:

In [ ]:
# We've already created the iskay_input dictionary in the previous section
# Now we just need to verify it's ready for use

print("Iskay Optimizer Configuration Ready:")
print("=" * 40)
print(f"  problem: QUBO with {len(iskay_input)} coefficients")
print(f"  problem_type: binary (0/1 variables)")
print(f"  backend_name: To be specified in Exercise 3")
print(f"  options: To be configured in Exercise 3")

<a id="exercise5"></a>
<div class="alert alert-block alert-success">
    
<b>Exercise 5: Configure and Run Iskay Optimizer (0 points)</b>

Configure the Iskay Optimizer with your credentials and submit the optimization job.

**Tasks:**
1. Complete the `iskay_input` dictionary with:
   - Your QUBO problem for instance `'ms_03_200_177.dat'`
   - Problem type should be `binary`
   - Backend name should be `"ibm_pittsburgh"`
   - Custom options with:
      - `num_iterations=5`
      - `transpilation_level=2`

2. Submit the job using `iskay_solver.run()`
3. Wait for the job to complete and retrieve results

</div>

<div class="alert alert-block alert-warning">
<b>Session Maximum Time</b>

The default session maximum time is set to 990 seconds (16.5 minutes). If your session exceeds this limit, any subsequent jobs will fail. Although it is possible to override this default session limit, please use the default timeout for the purpose of the QDC challenges to ensure fair access to the QPU for all participants. You should be able to achieve a competitive score within this time limit.

</div>

In [ ]:
### Write your code below here ###

problem = iskay_input  # Use the iskay_input dictionary we created earlier
problem_type = 
backend_name = 
options = 

### Don't change any code past this line ###

iskay_run_input = {
    "problem": problem,
    "problem_type": problem_type,
    "backend_name": backend_name,
    "options": options
}

In [ ]:
# Submit your answer using following code (check configuration before running on QPU)

grade_lab7_ex5(iskay_run_input) # Expected result type: dict

### Run the Quantum Optimization

Once your configuration passes the grader, we can submit our optimization problem to Kipu Quantum's infrastructure. The optimization process will use quantum algorithms to explore the solution space efficiently and find high-quality market assignments.

In [ ]:
# Submit the optimization job to Kipu Quantum's Iskay optimizer
print("Submitting optimization job to Kipu Quantum...")
print(f"Problem size: {A.shape[1]} variables, {len(iskay_input)} QUBO terms")

job = iskay_solver.run(**iskay_run_input)
job_id = job.job_id
print(f"Submitted to IBM with job ID: {job_id}")

Wait until the job is finished:

In [ ]:
while True:
    print(f"Waiting for job {job_id} to complete... (status: {job.status()})",  end='\r', flush=True)
    if job.status() in ['DONE', 'CANCELED', 'ERROR']:
        print(f"Job {job_id} completed with status: {job.status()}")
        break
    time.sleep(30)

print(f"\nJob {job_id} finished!")

# Retrieve the optimization results
result = job.result()
print("Optimization complete!")

### Understanding Quantum Optimization Output

The Iskay Optimizer returns comprehensive results that provide both quantum-level insights and business-actionable solutions.

#### Result Dictionary Structure

| Field | Type | Description |
|-------|------|-------------|
| `solution` | `Dict` | Variable assignments in the optimal solution |
| `solution_info` | `Dict` | Detailed information (bitstring, cost, mapping) |
| `prob_type` | `str` | Problem formulation type (binary or spin) |

#### Solution Info Dictionary

| Field | Type | Description |
|-------|------|-------------|
| `bitstring` | `str` | Optimal bitstring from quantum device |
| `cost` | `float` | Objective function value |
| `mapping` | `Dict[int, int]` | Bitstring to variable mapping |
| `seed_transpiler` | `int` | Transpiler seed for reproducibility |

In [ ]:
# Display the optimization results
print("Optimization Results")
print("=" * 50)
print(f"Problem Type: {result['prob_type']}")
print(f"\nSolution Info:")
print(f"  Bitstring: {result['solution_info']['bitstring']}")
print(f"  Cost: {result['solution_info']['cost']}")
print(f"\nSolution (first 10 variables):")
for i, (var, val) in enumerate(list(result['solution'].items())[:10]):
    print(f"  {var}: {val}")
print("  ...")

<a id="part-IV"></a>
## Part IV: Main Challenge — Tackling a Complex Market Split Instance (80 points)

Welcome to the main challenge of this notebook — the ultimate test of your quantum problem-solving skills!

<a id="exercise6"></a>
<div class="alert alert-block alert-success">
    
<b>Exercise 6: Tackling a Complex Market Splilt Instance (80 points)</b>

Your mission is to find a feasible solution for a complex instance of the Market Split problem — [`ms_05_100_003`](https://git.zib.de/qopt/qoblib-quantum-optimization-benchmarking-library/-/blob/main/01-marketsplit/instances/ms_05_100_003.dat?ref_type=heads) — from QOBLIB.  

Using the standard QUBO formulation and the QAOA algorithm, this instance demands 40 qubits and around thousands of two-qubit gates, pushing the boundaries of what today’s quantum hardware can handle. This is where your creativity and optimization skills truly come into play!

You are free to unleash any quantum techniques in your arsenal, including (but not limited to):  
- Qiskit Functions
- Custom QAOA implementations  
- Advanced transpilation or circuit optimization strategies  
- Alternative problem formulations
- Classical preprocessing or postprocessing tricks

See the [Quantum Optimization Best Practices repository](https://github.com/qiskit-community/qopt-best-practices) for ideas.

The reference solution for this instance is available in QOBLIB, but your goal is to design a general, scalable approach capable of solving problems of similar complexity (e.g., other `m=5` instances) with similar performance.  

> ⚠️ **Important:** Solutions that directly use or rely on the known answer will be disqualified. We’re looking for genuine quantum innovation — creative, algorithmic approaches that push the limits of what’s possible!

After the submission deadline, top scorers will have their notebooks reviewed by an expert panel, who will verify the results by running additional problem instances.  
Please refer to the [QDC contest rules](https://github.com/qiskit-community/qdc-challenges-2025/contest_rules.md) for more details on evaluation and submission procedures.

Now it’s your turn. Think outside the box, challenge the limits, and show what your quantum toolkit can do!

</div>

<div class="alert alert-block alert-warning">
    
<b>Score Formula</b>

The scoring function is defined as:

$$
\text{score} = 80 \times \left( 1 - \min\left(1, \frac{\text{violation}}{200}\right) \right)^4
$$

where `x` is the candidate solution, and `A` and `b` are the coefficient matrix and target vector defining the problem instance.  

The term **violation** measures how far the solution is from satisfying all constraints, and is computed as:

$$
\text{violation} = \sum_{i} \left( (A x - b)_i \right)^2
$$

A smaller violation indicates a more feasible solution. The score decreases rapidly as the violation grows, following a fourth-power penalty to strongly reward near-feasible solutions while heavily penalizing infeasible ones.  

If the violation exceeds **200**, the score is set to **zero**, while the maximum achievable score is **80**.
</div>

In [ ]:
### Write your code below here ###
# Following the Qiskit pattern as a suggestion

# Map the problem instance to quantum circuits and operators



# Optimize for target hardware



# Execute on target hardware



# Post-process results


# solution_bitstring is a binary string of length 40
solution_bitstring = 

### Don't change any code past this line ###

In [ ]:
# Submit your answer using following code

grade_lab7_ex6(solution_bitstring) # Expected result type: str

<a id="references"></a>
## References

[1] **Kipu Quantum Iskay Optimizer**  
IBM Quantum Functions Catalog. [https://docs.quantum.ibm.com/guides/functions](https://docs.quantum.ibm.com/guides/functions)

[2] **QOBLIB - Quantum Optimization Benchmarking Library**  
Zuse Institute Berlin (ZIB). [https://git.zib.de/qopt/qoblib-quantum-optimization-benchmarking-library](https://git.zib.de/qopt/qoblib-quantum-optimization-benchmarking-library)  
Repository of standardized benchmark instances for quantum and classical optimization algorithms.

[3] **Lodi, A., Tramontani, A., & Weninger, K. (2023)**  
"The Intractable Decathlon: Benchmarking Hard Combinatorial Problems."  
*INFORMS Journal on Computing*.  
Features the Market Split problem as one of ten challenging combinatorial optimization benchmarks.

[4] **Glover, F., Kochenberger, G., & Du, Y. (2019)**  
"Quantum Bridge Analytics I: A Tutorial on Formulating and Using QUBO Models."  
*4OR: A Quarterly Journal of Operations Research*, 17(4), 335-371.  
Comprehensive tutorial on QUBO formulations for optimization problems.

<a id="additional-information"></a>
## Additional Information

**Created by:** Murilo Henrique de Oliveira

**Advised by:** Junye Huang, Daniel Egger

**Version:** 1.0.0

<a id="package-versions"></a>
## Package Versions

In [ ]:
import qiskit
import qiskit_ibm_runtime
import qiskit_ibm_catalog

print(f'Qiskit: {qiskit.__version__}')
print(f'Qiskit IBM Runtime: {qiskit_ibm_runtime.__version__}')
print(f'Qiskit IBM Catalog: {qiskit_ibm_catalog.__version__}')
print(f'NumPy: {np.__version__}')